# UKB -- this notebook creates the df you will need to run the table 1 analyis
## you will need the files created in the previous three steps

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Download controls
# !dx download -r path/to/controls/
# !dx download -r path/to/cases/

#!dx download 'ukb_rel.dat' -- this is related info for UKB -- use cohort browser to create

# Start by selecting the NDD you would like to use
## you will need to run the notebook multiple times to create one df for each NDD

In [ ]:
#Select NDD
ndd = 'DEM'

# Combine cases and controls

In [ ]:
#Load cases
#cases = pd.read_csv(f'AD_cases_filtered.txt', sep = '\t')
cases = pd.read_csv('dementia_cases_ALL.txt', sep = '\t')
#cases = pd.read_csv('MS_cases.txt', sep = '\t')
#cases = pd.read_csv('PD_cases_filtered_with_dementia.txt', sep = '\t')
#cases = pd.read_csv('PD_cases_strict_no_dementia.txt', sep = '\t')
#cases = pd.read_csv('ALS_cases.txt', sep = '\t')
#cases = pd.read_csv('vascular_dementia_cases.txt', sep = '\t')
cases

In [ ]:
# Rename columns to match my code
#cases = cases.rename(columns = {'eid':'ID', f'{ndd}_date':f'{ndd}_DATE', 'sex':'GENETIC_SEX', 'townsend':'TOWNSEND', 'age_at_recruitment':'AGE_OF_RECRUIT'}) 
cases = cases.rename(columns = {'eid':'ID', f'Dementia_date':f'{ndd}_DATE', 'sex':'GENETIC_SEX', 'townsend':'TOWNSEND', 'age_at_recruitment':'AGE_OF_RECRUIT'}) 
cases = cases[['ID', 'GENETIC_SEX', 'TOWNSEND', 'AGE_OF_RECRUIT', f'{ndd}_DATE']]
cases = cases.sort_values(by = f'{ndd}_DATE')
cases

# Because our medication data only runs until the end of 2017, we are selecting NDD cases from the same time period (up to 2018)

In [ ]:
#Select cases from BEFORE 2018
cases[['year', 'month', 'day']] = cases[f'{ndd}_DATE'].str.split('-', expand = True)
cases['year'] = cases['year'].astype(int)
cases = cases[cases['year'] < 2018]
cases = cases.sort_values(by = f'{ndd}_DATE')
cases = cases[['ID', 'GENETIC_SEX', 'TOWNSEND', 'AGE_OF_RECRUIT', f'{ndd}_DATE']]
cases = cases.drop_duplicates(subset = 'ID', keep = 'first')
cases

In [ ]:
#Load controls
controls = pd.read_csv('NDD_healthy_controls_current_age_60.txt', sep = '\t')
controls[f'{ndd}_DATE'] = np.nan
controls = controls.rename(columns = {'eid':'ID', f'{ndd}_date':f'{ndd}_DATE', 'sex':'GENETIC_SEX', 'townsend':'TOWNSEND', 'age_at_recruitment':'AGE_OF_RECRUIT'}) 
controls = controls[['ID', 'GENETIC_SEX', 'TOWNSEND', 'AGE_OF_RECRUIT', f'{ndd}_DATE']]
controls = controls.drop_duplicates(subset = 'ID', keep = 'first')
controls

In [ ]:
# Combine cases and controls
df = pd.concat([cases, controls])

#Remove cases without TOWNEND 
df = df[~df['TOWNSEND'].isna()]

#Check to make sure no duplicate IDs
print(df.ID.value_counts())

df

In [ ]:
# Add BIRTH_YEAR -- this can be pulled using the cohort explorer if needed
b = pd.read_csv('birth_death_participant.csv')
b = b.rename(columns = {'eid':'ID', 'p34':'BIRTH_YEAR', 'p40000_i0':'DATE_OF_DEATH'})
b = b[['ID', 'BIRTH_YEAR']]
b

In [ ]:
#Merge with cases/controls
df = df.merge(b, left_on = 'ID', right_on = 'ID', how = 'left')
df

In [ ]:
# Add DEATH YEAR IF BEFORE 2018
d = pd.read_csv('birth_death_participant.csv')
d = d.rename(columns = {'eid':'ID', 'p34':'BIRTH_YEAR', 'p40000_i0':'DATE_OF_DEATH'})
d = d[['ID', 'DATE_OF_DEATH']]
d = d[~d['DATE_OF_DEATH'].isna()]
d = d.sort_values(by = 'DATE_OF_DEATH')
d[['year', 'month', 'day']] = d.DATE_OF_DEATH.str.split('-', expand = True)
d['year'] = d['year'].astype(int)
d = d[d['year'] < 2018]
d = d[['ID', 'DATE_OF_DEATH']]
d

In [ ]:
#Merge with cases/controls
df = df.merge(d, left_on = 'ID', right_on = 'ID', how = 'left')
df

In [ ]:
#Check for NAs
print('Sex:', df.GENETIC_SEX.isna().value_counts())
print('Birth year:', df.BIRTH_YEAR.isna().value_counts())
print('AGE:', df.AGE_OF_RECRUIT.isna().value_counts())
print('TOWNSEND:', df.TOWNSEND.isna().value_counts())

In [ ]:
#Eliminate IDs with no birth year
df = df[~df['BIRTH_YEAR'].isna()]
df

In [ ]:
#Check number of cases and controls
df[f'{ndd}_DATE'].isna().value_counts()

# Add drugs codes

In [ ]:
# Load list of people for whom we have drug data -- this file is created in step 01
p = pd.read_csv('drugs_april_2024_participant.csv')
participant_list = list(p['eid'])
print(len(participant_list))

In [ ]:
# Only select people from the have drug data list
df = df[df['ID'].isin(participant_list)]
df

In [ ]:
df[f'{ndd}_DATE'].isna().value_counts()

In [ ]:
# We came up with this list of drugs in notebook 03
drugs_list = ['moxifloxacin', 'fexofenadine', 'lormetazepam', 'nizatidine', 'lacidipine', 'rifaximin', 'levomepromazine', 'losartan', 'metformin', 'colchicine', 'valproicacid', 'acetylcysteine', 'phenoxymethylpenicillin', 'erdosteine', 'montelukast', 'sildenafil', 'amiodarone', 'clomethiazole', 'gemfibrozil', 'alogliptin', 'mirtazapine', 'aclidiniumbromide', 'clomipramine', 'nebivolol', 'minoxidil', 'clonidine', 'alfuzosin', 'rizatriptan', 'mecysteine', 'rimonabant', 'naratriptan', 'paracetamolandmetoclopramide', 'orlistat', 'griseofulvin', 'telmisartan', 'pravastatin', 'zaleplon', 'piroxicam', 'apixaban', 'amphotericin', 'linagliptin', 'amlodipine', 'diltiazem', 'nortriptyline', 'potassiumchlorideandpotassiumbicarbonate', 'propantheline', 'hydrotalcite', 'alprazolam', 'indomethacin', 'ciclesonide', 'nabumetone', 'perindopril', 'digoxin', 'oxerutins', 'amisulpride', 'bisacodyl', 'carbocisteine', 'gabapentin', 'misoprostol', 'ticagrelor', 'apomorphine', 'tetrabenazine', 'tetracycline', 'nalidixicacid', 'methylcellulose', 'flucloxacillin', 'epogam', 'mefenamic', 'dantrolene', 'cyproheptadine', 'phenylephrine', 'terbutaline', 'fentanyl', 'fluoxetine', 'moxisylyte', 'zonisamide', 'doxepin', 'sodiumvalproate', 'thioridazine', 'clomifene', 'fesoterodine', 'clonazepam', 'orciprenaline', 'fusidicacid', 'metronidazole', 'tramadol', 'ibandronicacid', 'gliclazide', 'naltrexone', 'chlortalidone', 'ciprofloxacin', 'orphenadrine', 'sulpiride', 'aminophylline', 'aspirin', 'glimepiride', 'budesonide', 'eprosartan', 'carvedilol', 'rupatadine', 'acemetacin', 'ibuprofen', 'aceclofenac', 'ketorolac', 'glyceryltrinitrate', 'olsalazine', 'zafirlukast', 'flurbiprofen', 'trimethoprim', 'cefixime', 'fosinopril', 'inositolnicotinate', 'entacapone', 'lercanidipine', 'acetazolamide', 'indapamide', 'metoprolol', 'febuxostat', 'captopril', 'disulfiram', 'propylthiouracil', 'tolbutamide', 'ezetimibe', 'risedronate', 'bisoprolol', 'pantoprazole', 'flavoxate', 'cinnarizine', 'nateglinide', 'hydrochlorothiazideandramipril', 'magnesium', 'eletriptan', 'trifluoperazine', 'chloramphenicol', 'alimemazine', 'moxonidine', 'oxcarbazepine', 'cetirizine', 'pseudoephedrine', 'albuterol', 'hydroxychloroquine', 'trimipramine', 'trimethoprimandsulfamethoxazole', 'nifedipine', 'sotalol', 'hyoscinebutylbromide', 'loratadine', 'erythromycin', 'naftidrofuryl', 'modafinil', 'esomeprazole', 'cisapride', 'oxytetracycline', 'tinidazole', 'sumatriptan', 'calcitriol', 'ranitidine', 'leflunomide', 'famciclovir', 'oxybutynin', 'perphenazine', 'ropinirole', 'ursodeoxycholicacid', 'citalopram', 'riluzole', 'darifenacin', 'buspirone', 'alendronatesodium', 'theophylline', 'paracetamolandcodeine', 'sulfasalazine', 'pre-exposureprophylaxis', 'lamotrigine', 'dexamethasone', 'stalevo', 'norfloxacin', 'ofloxacin', 'mianserin', 'nitrofurantoin', 'memantine', 'lorazepam', 'anticholinergic', 'midazolam', 'dronedarone', 'enalapril', 'fluticasone', 'lithium', 'mebendazole', 'diclofenac', 'quinapril', 'itraconazole', 'pyridostigmine', 'methylphenidate', 'trihexyphenidyl', 'co-amilofruse', 'mizolastine', 'irbesartan', 'glipizide', 'acamprosate', 'cyproterone', 'methotrexate', 'tolfenamicacid', 'prasugrel', 'mefloquine', 'methylprednisolone', 'isotretinoin', 'ibuprofenandcodeine', 'letrozole', 'nicardipine', 'magnesiumhydroxide', 'vancomycin', 'ipratropium', 'prednisone', 'potassiumchloride', 'saxagliptin', 'risperidone', 'dipipanone', 'tadalafil', 'astemizole', 'simvastatin', 'acarbose', 'quinine', 'levothyroxine', 'triamterene', 'nystatin', 'chlordiazepoxide', 'pramipexole', 'dihydrocodeine', 'allopurinol', 'terfenadine', 'mesalazine', 'hydrocortisonebuccaltablets', 'doxycycline', 'trandolapril', 'dicyclomine', 'asilone', 'nadolol', 'frovatriptan', 'ranolazine', 'azithromycin', 'cholestyramine', 'zopiclone', 'vigabatrin', 'tiotropium', 'bromocriptine', 'tiaprofenicacid', 'lidocaine', 'phenelzine', 'levofloxacin', 'diflunisal', 'hydralazine', 'acitretin', 'metoclopramideandaspirin', 'ciclosporin', 'methyldopa', 'tripotassiumdicitratobismuthate', 'galantamine', 'cyclobenzaprine', 'codeine', 'rifampicinandisoniazid', 'valaciclovir', 'phenytoin', 'co-amilozide', 'etoricoxib', 'sulindac', 'liothyronine', 'atenolol', 'cinacalcet', 'co-beneldopa', 'lansoprazole', 'indoramin', 'fenofibrate', 'zolmitriptan', 'hydrochlorothiazideandlisinopril', 'trospium', 'sucralfate', 'etamsylate', 'eplerenone', 'cromolynsodium', 'benzydamine', 'atorvastatin', 'cefalexin', 'minocycline', 'hydroxycarbamide', 'pioglitazone', 'rasagiline', 'carbamazepine', 'pivmecillinam', 'isosorbidedinitrate', 'formoterol', 'pentoxifylline', 'cabergoline', 'diazepam', 'furosemide', 'amiloride', 'tranexamicacid', 'ramipril', 'lofepramine', 'etidronicacid', 'promazine', 'aliskren', 'penicillamine', 'chlorpheniramine', 'prochlorperazine', 'bismuthsubsalicylate', 'cyclopenthiazide', 'finasteride', 'ironandfolicacid', 'penicillin', 'cyclizine', 'sertraline', 'ampicillin', 'dexketoprofen', 'nefazodone', 'solifenacin', 'felodipine', 'fluvoxamine', 'bupropion', 'tramacet', 'amoxicillin', 'tolterodine', 'isosorbidemononitrateandisosorbidedinitrate', 'betamethasone', 'cefradine', 'co-amoxiclav', 'clarithromycin', 'indometacin', 'almotriptan', 'escitalopram', 'chloroquine', 'ketotifen', 'oseltamivir', 'tamsulosin', 'olanzapine', 'levocetirizine', 'fluphenazine', 'chloralhydrate', 'tizanidine', 'cerivastatin', 'pholcodine', 'bezafibrate', 'methadone', 'bendroflumethiazide', 'methocarbamol', 'rivaroxaban', 'dosulepin', 'verapamil', 'mirabegron', 'salbutamol', 'amitriptyline', 'omeprazole', 'baclofen', 'fishoil', 'venlafaxine', 'meptazinol', 'cefpodoxime', 'fludrocortisone', 'levetiracetam', 'rosiglitazone', 'isoniazid', 'empagliflozin', 'carbimazole', 'lisinopril', 'doxazosin', 'co-magaldrox', 'temazepam', 'imipramine', 'gaviscon', 'mycophenolicacid', 'cefadroxil', 'methenamine', 'nicorandil', 'loperamide', 'mometasone', 'sterculia', 'cimetidine', 'hydroxyurea', 'celecoxib', 'meperidine', 'imidapril', 'terazosin', 'atropine', 'flecainide', 'thyroxine', 'hyoscinehydrobromide', 'dantronanddocusate', 'dabigatran', 'fluconazole', 'prucalopride', 'mebeverine', 'macrogol', 'lactulose', 'glibenclamide', 'sibutramine', 'meloxicam', 'valdecoxib', 'manevac', 'topiramate', 'ketoprofen', 'primidone', 'paroxetine', 'acrivastine', 'sodiumfusidate', 'strontium', 'zolpidem', 'oxprenolol', 'sulfinpyrazone', 'carisoprodol', 'pizotifen', 'clemastine', 'rivastigmine', 'bumetanide', 'agomelatine', 'famotidine', 'reboxetine', 'pregabalin', 'alprostadil', 'haloperidol', 'candesartan', 'kaolin', 'ivabradine', 'linaclotide', 'atenololandnifedipine', 'repaglinide', 'selegiline', 'olmesartan', 'oxazepam', 'paracetamol', 'dutasteride', 'prazosin', 'sodiumcromoglicate', 'amantadine', 'nefopam', 'mercaptopurine', 'insulin', 'docusate', 'buprenorphine', 'isosorbidemononitrate', 'chlorphenamine', 'diphenhydramine', 'valganciclovir', 'azathioprine', 'proguanil', 'propranolol', 'prednisolone', 'immunoglobin', 'metoclopramide', 'celiprolol', 'rofecoxib', 'trazodone', 'tamoxifen', 'raloxifene', 'moclobemide', 'metolazone', 'omega-3', 'dipyridamole', 'alverine', 'alendronicacid', 'quetiapine', 'duloxetine', 'simeticone', 'beclometasone', 'azapropazone', 'lymecycline', 'aripiprazole', 'domperidone', 'rabeprazole', 'hydrocortisone', 'phenobarbital', 'betahistine', 'salmeterol', 'hydroxyzine', 'loprazolam', 'prilocaine', 'triamcinolone', 'zoledronicacid', 'anastrozole', 'fybogel', 'canagliflozin', 'clopidogrel', 'balsalazide', 'vardenafil', 'lumiracoxib', 'labetalol', 'brompheniramine', 'danazol', 'clodronicacid', 'warfarin', 'methysergide', 'hydrochlorothiazide', 'sodiumpicosulfate', 'ondansetron', 'adalimumab', 'rifampicin', 'tenoxicam', 'colesevelam', 'ciprofibrate', 'chlorpromazine', 'pancrelipase', 'co-danthrusate', 'disopyramide', 'oxycodone', 'valsartan', 'nitrazepam', 'tacrolimus', 'lacosamide', 'flupentixol', 'cyclophosphamide', 'nitrousoxide', 'ethambutol', 'rosuvastatin', 'clobazam', 'donepezil', 'promethazine', 'spironolactone', 'co-careldopa', 'clindamycin', 'naproxen', 'senna', 'cefaclor', 'fluvastatin', 'propiverine', 'timolol', 'dutasterideandtamsulosin', 'dapagliflozin', 'migril', 'pilocarpine', 'aciclovir', 'morphine', 'sevelamer', 'etodolac', 'dapsone', 'sitagliptin', 'nedocromil', 'propafenone', 'oxitropium', 'cefuroxime']
print(len(drugs_list))

In [ ]:
# download the csv files of drug data if needed (see step 03)
#! dx download -r csv_files

In [ ]:
df

In [ ]:
# Add drug data to df
for drug in drugs_list:
    code = pd.read_csv(f'/path/to/drugs/{drug}_with_date.csv')
    code = code[['ID', f'{drug}_DATE']]
    df = df.merge(code, left_on = 'ID', right_on = 'ID', how = 'left')

In [ ]:
# Create list of headers drug + DATE
codes = []
for drug in drugs_list:
    a = drug + '_DATE'
    codes.append(a)
    
print(len(codes))

# Prep for COX

In [ ]:
#Set variables
STUDY_ENDS = '2018-01-01'

In [ ]:
#Drop duplicates
df = df.sort_values(by = f'{ndd}_DATE')
df = df.drop_duplicates(subset = 'ID', keep = 'first')
print(len(df))
df.head()

In [ ]:
#Calculate the year they were recruited to the study
df['recruit_year'] = df['BIRTH_YEAR']+df['AGE_OF_RECRUIT']

In [ ]:
#Create tenure for people with NDD - select folks with an NDD date
has_NDD = df[~df[ndd + '_DATE'].isna()]

#Calculate the tenure, i.e. the time from the beginning of the study to their time of NDD diagnosis
has_NDD['tenure'] = (pd.to_datetime(has_NDD[ndd + '_DATE'], errors = 'coerce') - pd.to_datetime(has_NDD['recruit_year'], format='%Y')).dt.days/365

#Only keep people who got NDD after they joined study
has_NDD = has_NDD[has_NDD['tenure'] > 0]

#Add age_at_tenure to people with an NDD
has_NDD['age_at_tenure'] = (pd.to_datetime(has_NDD[ndd + '_DATE'], errors = 'coerce') - pd.to_datetime(has_NDD['BIRTH_YEAR'], format='%Y')).dt.days/365

#Create tenure for people without NDD - select folks with no NDD date
NDD_free = df[df[ndd + '_DATE'].isna()]

#For people without NDD, break into dead and alive
alive = NDD_free[NDD_free['DATE_OF_DEATH'].isna()]
dead = NDD_free[~NDD_free['DATE_OF_DEATH'].isna()]

#Calculate the tenure for people who are still alive, i.e. the time from the beginning of the study to the end of study
alive['tenure'] = (pd.to_datetime(STUDY_ENDS) - pd.to_datetime(alive['recruit_year'], format='%Y')).dt.days/365

#Add age_at_tenure for people who are still alive
alive['age_at_tenure'] = (pd.to_datetime(STUDY_ENDS) - pd.to_datetime(alive['BIRTH_YEAR'], format='%Y')).dt.days/365

#Calculate the tenure for people who are still dead, i.e. the time from the beginning of the study to the end of study
dead['tenure'] = (pd.to_datetime(dead['DATE_OF_DEATH']) - pd.to_datetime(dead['recruit_year'], format='%Y')).dt.days/365

#Add age_at_tenure for people who are dead
dead['age_at_tenure'] = (pd.to_datetime(dead['DATE_OF_DEATH']) - pd.to_datetime(dead['BIRTH_YEAR'], format='%Y')).dt.days/365

#Combine two groups
df = pd.concat([has_NDD, alive, dead])

In [ ]:
#Encode NDD to 1 or 0
df[ndd] = np.where(df[ndd + '_DATE'].isna(), 0, 1)

#GENETIC_SEX to 1 or 2
df.loc[df.GENETIC_SEX == 'Female', 'GENETIC_SEX'] = '2'
df.loc[df.GENETIC_SEX == 'Male', 'GENETIC_SEX'] = '1'

In [ ]:
df

In [ ]:
for code in drugs_list:
    df['Lag_' + code] = (pd.to_datetime(df[code + '_DATE'], errors = 'coerce') - pd.to_datetime(df['recruit_year'], format = '%Y')).dt.days/365
        
    #Select data if it happened before study end -- lag 0
    df['QC0_' + code] = np.where((df['Lag_' + code] < df['tenure']), 1, 0)

In [ ]:
#Find related individuals
related = pd.read_csv('ukb_rel.dat', sep = ' ')
related = related[related['Kinship']> 0.0884]
related

In [ ]:
#Create list of all people in our df with NDD status for analysis
test = df[['ID', f'{ndd}']]
test

In [ ]:
#Create list of all people in our df
ndd_ids = list(df['ID'])

#Create list of people who are related & in our df
remove = related.loc[related['ID1'].isin(ndd_ids) & related['ID2'].isin(ndd_ids)]
remove

In [ ]:
remove = remove.merge(test, left_on = 'ID1', right_on = 'ID', how = 'left')
remove = remove.rename(columns = {f'{ndd}':f'{ndd}1'})
remove = remove.merge(test, left_on = 'ID2', right_on = 'ID', how = 'left')
remove = remove.rename(columns = {f'{ndd}':f'{ndd}2'})
remove

In [ ]:
#This prioritizes people without an NDD
remove_list = []
for i in range (len(remove)):
    if remove[f'{ndd}1'][i] == 0:
        remove_list.append(remove['ID1'][i])
    else:
        remove_list.append(remove['ID2'][i])

In [ ]:
print(len(remove_list))

In [ ]:
df_final = df[~df['ID'].isin(remove_list)]
df_final

In [ ]:
removed_df = df[df['ID'].isin(remove_list)]
removed_df[f'{ndd}_DATE'].isna().value_counts()

In [ ]:
df_final[f'{ndd}_DATE'].isna().value_counts()

In [ ]:
df_final

In [ ]:
df_final.to_csv(f'{ndd}_with_tenure_OCT_09_24.csv', header = True, index = None)

In [ ]:
print(f'!dx upload {ndd}_with_tenure_OCT_09_24.csv --path /data/files_for_cox/{ndd}_with_tenure_OCT_09_24.csv')

In [ ]:
# save created dataframe

In [ ]:
# Print numbers for figure 1 here
import pandas as pd
# # #ndd_list = ['AD']
ndd_list = ['AD', 'PD', 'DEM']

for ndd in ndd_list:
    test = pd.read_csv(f'{ndd}_with_tenure_OCT_09_24.csv', low_memory = False)
    print(ndd)
    cases = test[~test[f'{ndd}_DATE'].isna()]
    controls = test[test[f'{ndd}_DATE'].isna()]
    print('Cases: ', len(cases))
    print('Controls: ', len(controls))
    female = cases[cases['GENETIC_SEX'] == 2]
    male = cases[cases['GENETIC_SEX'] == 1]
    print("Female: ", len(female))
    print("Male: ", len(male))
    print(len(female) + len(male))
    
    print("Controls by gender:")
    cases = controls
    female = cases[cases['GENETIC_SEX'] == 2]
    male = cases[cases['GENETIC_SEX'] == 1]
    print("Female: ", len(female))
    print("Male: ", len(male))
    print(len(female) + len(male))
    
    print('\n')